In [145]:
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

import numpy as np

import pandas as pd


In [135]:
# carregar modelo word2vec
word2vec = KeyedVectors.load_word2vec_format('cbow_s50.txt')
vocab_size, embedding_size = word2vec.vectors.shape

In [171]:
# carregar dados de treinamento do CSV
df = pd.read_csv('train.csv', header=None, names=['texto', 'classe'])
X = df['texto'].values
y = df['classe'].values

In [172]:
display(X)

array(['Endereço dos Campus',
       'Qual é o endereço do campus CCET da UNIRIO?',
       'Qual é o endereço do campus CCH da UNIRIO?', ...,
       'Como realizar matrícula em disciplina específica?',
       'Em disciplina específica como realizar matrícula?',
       'Como em disciplina específica realizar matrícula?'], dtype=object)

In [173]:
# tokenizar o texto
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# padronizar o tamanho das sequencias
max_length = max([len(text) for text in X])
X = pad_sequences(X, maxlen=max_length, padding='post')

In [174]:
# construir modelo
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, weights=[word2vec.vectors], input_length=max_length, trainable=False))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(7, activation='softmax'))

In [175]:
# compilar modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [176]:
# separar dados em treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, 7)
y_val = to_categorical(y_val, 7)

In [144]:
# treinar modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
26/26 [==============================] - 3s 35ms/step - loss: 1.5030 - accuracy: 0.4729 - val_loss: 0.7299 - val_accuracy: 0.8177
Epoch 2/10
26/26 [==============================] - 0s 11ms/step - loss: 0.4516 - accuracy: 0.8670 - val_loss: 0.2393 - val_accuracy: 0.9212
Epoch 3/10
26/26 [==============================] - 0s 11ms/step - loss: 0.1774 - accuracy: 0.9594 - val_loss: 0.1467 - val_accuracy: 0.9507
Epoch 4/10
26/26 [==============================] - 0s 11ms/step - loss: 0.0919 - accuracy: 0.9766 - val_loss: 0.2030 - val_accuracy: 0.9360
Epoch 5/10
26/26 [==============================] - 0s 12ms/step - loss: 0.0681 - accuracy: 0.9840 - val_loss: 0.0922 - val_accuracy: 0.9655
Epoch 6/10
26/26 [==============================] - 0s 11ms/step - loss: 0.0405 - accuracy: 0.9914 - val_loss: 0.1279 - val_accuracy: 0.9655
Epoch 7/10
26/26 [==============================] - 0s 11ms/step - loss: 0.0417 - accuracy: 0.9914 - val_loss: 0.0896 - val_accuracy: 0.9754
Epoch 8/10
26